In [28]:
pip install rouge

In [29]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from rouge import Rouge

# hyperparameters
batch_size = 32 # how many independent sequences
block_size = 8 # what is the maximum context length
max_iters = 5000
eval_interval = 500
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32
n_head = 4

torch.manual_seed(1337)


with open('/content/input (1).txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        perplexities = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss, perplexity = model(X, Y)
            losses[k] = loss.item()
            perplexities[k] = perplexity.item() if perplexity is not None else float('nan')
        out[f'{split}_loss'] = losses.mean()
        out[f'{split}_perplexity'] = perplexities.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self,head_size):
      super().__init__()
      self.key=nn.Linear(n_embd,head_size,bias=False) ##Corresponds to the embedding layer, (see BTC shape here C is the embedding vector)
      self.query=nn.Linear(n_embd,head_size,bias=False)
      self.value=nn.Linear(n_embd,head_size,bias=False)
      #Here tril is not the part of nn module to we add tril as buffer, meaning our custom function
      self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size))) ##we made an upper triangle matrix above of shape T,T (8,8)

#Forward process for the computation here
    def forward(self,x):
      B,T,C=x.shape
      k=self.key(x)
      q=self.query(x)

      weight=q @ k.transpose(-2,-1) * C**-0.5
      weight=weight.masked_fill(self.tril[:T,:T]==0,float('-inf'))
      weight=F.softmax(weight,dim=-1)
      v=self.value(x)
      output=weight@v
      return output

class MultiHeadAttention(nn.Module):
    def __init__(self,num_head,head_size):
      super().__init__()
      self.head=nn.ModuleList([Head(head_size) for _ in range(num_head)])
      #Adding a projection
      self.proj=nn.Linear(n_embd,n_embd)



    def forward(self,x):
      output=torch.cat([h(x) for h in self.head],dim=-1) # AS THE CONCAT TO THE CHANNEL SIDE SO DIM=-1(C)
      output=self.proj(output)
      return output

class FeedForward(nn.Module):
  def __init__(self, n_embd):
    super().__init__()
    self.net=nn.Sequential(
        nn.Linear(n_embd,4* n_embd),
        nn.ReLU(),
        nn.Linear(4* n_embd,n_embd)
    )

  def forward(self,x):
    return self.net(x)

'''
Adding the block here: with residual conections to optimzie and make GD faster.
Looking into the paper of the residual connection. basically adding the initial feature to lmake the model
learn the forgotten features etc.
'''

class Block(nn.Module):
  def __init__(self,n_embd,n_heads):
    super().__init__()
    self.con_head=MultiHeadAttention(n_heads,n_embd//n_heads)
    self.ffdnn=FeedForward(n_embd)
    self.ln_1=nn.LayerNorm(n_embd)
    self.ln_2=nn.LayerNorm(n_embd)


  def forward(self,x):
      x=x+self.con_head(self.ln_1(x))
      x=x+self.ffdnn(self.ln_2(x))
      return x



# super simple bigram model-5
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks=nn.Sequential(
            Block(n_embd,n_heads=4),
            Block(n_embd,n_heads=4),
            Block(n_embd,n_heads=4),
            nn.LayerNorm(n_embd),

        )
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb +pos_emb # (B,T,C)
        x=self.blocks(x)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss=None
            perplexity=None
        else:
            B,T,C =logits.shape
            logits=logits.view(B*T, C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits, targets)
            probabilities=F.softmax(logits, dim=-1)
            log_probabilities=torch.log(probabilities)
            log_prob=torch.gather(log_probabilities,1,targets.unsqueeze(1)).squeeze(1)
            N=targets.size(0)
            perplexity=torch.exp(-torch.sum(log_prob)/N)

        return logits, loss,perplexity

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond =idx[:, -block_size:]
            # get the predictions
            logits, loss,perplexity = self(idx_cond)
            # focus only on the last time step
            logits =logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs =F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next =torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx =torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train_loss']:.4f}, val loss {losses['val_loss']:.4f}, PERPLEX_TRAIN {losses['train_perplexity']:.4f} "+
              f"PERPLEX_VAL {losses['val_perplexity']:.4f}"
        )

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss,perplexity = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

print("#########-----------------------------------------------------------------------------------------------------------------------------###### \n \n")
rouge = Rouge()
generated_summaries = []
context = torch.tensor([[4,24,26]], dtype=torch.long, device=device)
summary= decode(m.generate(context, max_new_tokens=2000)[0].tolist())
generated_summaries.append(summary)
reference_summaries = []
reference_summaries.append(text[:2000])
print("REFERENCE SUMMARY: \n"+str(reference_summaries[0][:50])+"\n")
print("GENERATED SUMMARY: \n"+str(generated_summaries[0][:50])+"\n")
print("ROUGE_SCORES: \n")
rouge_scores = rouge.get_scores(generated_summaries, reference_summaries)
for score in rouge_scores:
    print(score)

0.042369 M parameters
step 0: train loss 4.3103, val loss 4.3097, PERPLEX_TRAIN 74.5176 PERPLEX_VAL 74.4740
step 500: train loss 2.3998, val loss 2.4008, PERPLEX_TRAIN 11.0578 PERPLEX_VAL 11.0641
step 1000: train loss 2.2644, val loss 2.2660, PERPLEX_TRAIN 9.6623 PERPLEX_VAL 9.6832
step 1500: train loss 2.1670, val loss 2.1896, PERPLEX_TRAIN 8.7668 PERPLEX_VAL 8.9721
step 2000: train loss 2.1318, val loss 2.1680, PERPLEX_TRAIN 8.4632 PERPLEX_VAL 8.7813
step 2500: train loss 2.0813, val loss 2.1305, PERPLEX_TRAIN 8.0435 PERPLEX_VAL 8.4553
step 3000: train loss 2.0526, val loss 2.1242, PERPLEX_TRAIN 7.8206 PERPLEX_VAL 8.4115
step 3500: train loss 2.0436, val loss 2.1059, PERPLEX_TRAIN 7.7490 PERPLEX_VAL 8.2497
step 4000: train loss 2.0137, val loss 2.0946, PERPLEX_TRAIN 7.5212 PERPLEX_VAL 8.1520
step 4500: train loss 1.9908, val loss 2.0944, PERPLEX_TRAIN 7.3469 PERPLEX_VAL 8.1607
step 4999: train loss 1.9801, val loss 2.0589, PERPLEX_TRAIN 7.2689 PERPLEX_VAL 7.8780

Row duke bergeer?

V